In [ ]:
import sympy
from control.matlab import *
from IPython.display import Latex, display
from sympy import Poly
from sympy.abc import s, z

In [1]:
%pylab %matplotlib inline

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


<div id="toc"></div>

# DC Motor Position: Frequency Domain Methods for Controller Design

From the main problem, the open-loop transfer function of the DC Motor is given as follows.

$$
P(s) = \frac {\Theta (s)}{V(s)} = \frac{K}{s ( (Js + b)(Ls + R) + K^2 )} \qquad [ \frac{rad}{V}  ]
$$

The structure of the control system has the form shown in the figure below.
![feedback_motorp.png](figures/feedback_motorp.png)
For the original problem setup and the derivation of the above equations, please refer to the [DC Motor Position: System Modeling](../MotorPosition/MotorPosition-SystemModeling.ipynb) page. With a 1-radian step reference, the design criteria are the following.
* Settling time less than 0.040 seconds
* Overshoot less than 16
* No steady-state error, even in the presence of a step disturbance input First create a new [m-file](../Extras_Mfile.ipynb) and type in the following commands (refer to main problem for the details of getting these commands).

In [2]:
# J = 3.2284E-6;
# b = 3.5077E-6;
# K = 0.0274;
# R = 4;
# L = 2.75E-6;
# s = tf('s');
# P_motor = K/(s*((J*s+b)*(L*s+R)+K^2));

## Drawing the uncompensated system's Bode plot

The main idea of frequency-based design is to use the Bode plot of the open-loop transfer function to estimate the system's closed-loop response. Adding a controller to the system changes the open-loop Bode plot, thereby changing the closed-loop response. Let's first draw the Bode plot for the original open-loop transfer function. Throughout this section, we will use the interactive **SISO Design Tool** to design our controller. This tool can be opened for the uncompensated plant by adding the command |sisotool('bode',P_motor)| to your m-file. Two windows will open initially with the **SISO Design for SISO Design Task** window having the form shown in the figure below. This figure shows the open-loop Bode plot of the transfer function passed via the |sisotool| function. This figure appears initially due to the addition of the string |'bode'| to the function call. If the string |'bode'| is omitted, the default initial window includes the root locus plot in addition to the open-loop Bode plot.
![Picture1.png](figures/Picture1.png)
From previous pages we know that the open-loop plant in unity feedback does not satisfy the given system requirements. We will, therefore, add different compensators to our system in order to reshape the open-loop Bode plot to give the desired closed-loop performance.

## Adding an integrator

Now let's add a pure integrator compensator to our system. The architecture of the feedback system can be modified within the **SISO Design Tool** GUI, but the default compensator **C** is placed in series with the plant **G** in the forward path of the block diagram. This corresponds to the architecture we are assuming, as shown in the schematic at the top of this page. Placing an integral compensator in this position will achieve the goal of zero steady-state error in the presence of a step disturbance input. The form of our controller is then modified under the **Compensator Editor** tab of the **Control and Estimation Tools Manager** window. Specifically, an integrator is added to our controller by right-clicking in the **Dynamics** section of the window and selecting *Add Pole/Zero > Integrator* from the resulting menu. The figure below illustrates what the **Control and Estimation Tools Manager** window should look like.
![Picture3a.png](figures/Picture3a.png)
The Bode plot that is already open should then change as shown below to reflect the addition of the integrator.
![Picture3.png](figures/Picture3.png)


## Gain and phase margin specifications and controller design

Recall that we additionally desire an overshoot of less than 16%. Overshoot of a step response correlates to a system's phase margin. We will apply the relationship that phase margin approximately equals 100 times the damping ratio (_zeta_) to estimate the desired phase margin of our system. We will additionally use our 0.040 second settling time requirement to estimate the desired bandwidth frequency (_Wbw_) of our closed-loop system. Add the following code to your m-file to estimate the desired phase margin and bandwidth. Note that these relationships are intended for a second-order canonical system, which we do not have, but the results will serve as a good starting point for our design.

In [3]:
# zeta = -log(.16) / sqrt( pi^2 + (log(.16))^2 );
# PM = 100*zeta
# Wbw = (4/(0.04*zeta))*sqrt((1 - 2*zeta^2) + sqrt(4*zeta^4 - 4*zeta^2+2))

The above results demonstrate that we desire to have at least 50 degrees of phase margin in an attempt to meet the overshoot requirement. Also, a closed-loop bandwidth of 250 rad/sec is desired to additionally achieve the settle time requirement. We will estimate the closed-loop bandwidth as the point on the open-loop Bode plot where the magnitude is between -6 and -7.5 dB. From the Bode plot we see that we must add about 110 degrees of phase and 80 dB of gain at a frequency of 250 rad/sec in order to move the gain crossover frequency to 250 rad/sec and provide 50 degrees of phase margin. The magnitude plot will then also lie between -6 and -7.5 dB at a frequency greater than 250 rad/sec. From the Bode phase plot we can see that there is a pole near -60 as indicated by the blue |x| on the Bode plot near the frequency 60 rad/sec (as well as the change in the magnitude and phase plots). We will, therefore, begin to modify our compensator by adding a zero at _s_ = -60 in order to flatten out the phase curve. A real zero can be added again by right-clicking in the **Dynamics** section of the **Compensator Editor** tab of the **Control and Estimation Tools Manager** window. The location of the zero can then be set by typing -60 into the **Location** cell of the **Edit Selected Dynamics** portion of the window and pressing **Enter**. Your compensator now has the form of a PI controller and the compensated open-loop Bode plot should have the modified form shown below where the red |o| shows the location of the new zero. To achieve the limits on the axes shown below, right-click on the Bode plot and select **Properties** from the resulting menu.
![Picture5b.png](figures/Picture5b.png)
From the Bode plot we can see that we need at least 50 additional degrees of phase if we move our crossover frequency to 250 rad/sec. Let's then add a lead compensator to add exactly 50 degrees of phase at this frequency. The following equations provide the pole and zero locations of the lead compensator.

In [4]:
# a = (1 - sin(PM*pi/180))/(1 + sin(PM*pi/180));
# T = 1/(Wbw*sqrt(a));
# zero = -1/T
# pole = -1/(a*T)

In [5]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>